In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
pip install pyspark

     |████████████████████████████████| 281.3 MB 34 kB/s 
     |████████████████████████████████| 198 kB 60.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=d69d0d63211363181a71e0906f5c15db3abc695242dfdde669945b4c90f325be
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Dataframe').getOrCreate()
spark

In [ ]:
## read the dataset
df_cards=spark.read.option('header','true').option('delimiter','|').csv('/content/drive/MyDrive/DatosCollabBigData/cards.csv',inferSchema=True)
df_cards.show(5)

+----------+-----------+------------+----------+--------------+-------+------+
|CP_CLIENTE|CP_COMERCIO|      SECTOR|       DIA|FRANJA_HORARIA|IMPORTE|NUM_OP|
+----------+-----------+------------+----------+--------------+-------+------+
|      4007|       4006|ALIMENTACION|2015-09-05|         08-10| 297.05|    10|
|      4007|       4006|ALIMENTACION|2015-04-20|         14-16| 494.23|    17|
|      4007|       4006|ALIMENTACION|2015-06-04|         10-12|1694.15|    57|
|      4002|       4009|ALIMENTACION|2015-12-05|         20-22|  79.71|     2|
|      4002|       4007|ALIMENTACION|2015-03-07|         20-22| 181.28|     3|
+----------+-----------+------------+----------+--------------+-------+------+
only showing top 5 rows



In [ ]:
df_weather=spark.read.option('header','true').option('delimiter',';').csv('/content/drive/MyDrive/DatosCollabBigData/weather.csv',inferSchema=True)
df_weather.show(5)

+----------+---+----+-----+----+-----+----+------+------+------+---------+---------+----+------+----+
|     FECHA|DIA|TMax|HTMax|TMin|HTMin|TMed|HumMax|HumMin|HumMed|VelViento|DirViento| Rad|Precip| ETo|
+----------+---+----+-----+----+-----+----+------+------+------+---------+---------+----+------+----+
|2015-12-30|365|19.2|12:02| 8.6| 4:06|13.9|  98.8|  49.3|  71.4|      0.6|    337.2| 9.9|   0.0|1.04|
|2015-12-29|363|18.8|12:38|10.4| 4:32|14.2| 100.0|  64.7|  81.2|      1.0|    292.7| 8.5|   0.0|1.07|
|2015-12-29|364|18.8|12:14|11.0|22:22|14.4|  89.3|  55.6|  70.2|      1.2|    328.2|10.0|   0.0|1.32|
|2015-12-28|362|18.9|10:00|11.6|23:48|15.8|  89.0|  47.4|  66.1|      1.2|    280.8| 9.3|   0.0|1.38|
|2015-12-27|361|19.6|13:48|12.5| 6:30|15.8|  72.9|  48.3|  61.1|      2.0|     35.8| 9.4|   0.0|1.98|
+----------+---+----+-----+----+-----+----+------+------+------+---------+---------+----+------+----+
only showing top 5 rows



In [ ]:
df_cards.printSchema()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

root
 |-- CP_CLIENTE: integer (nullable = true)
 |-- CP_COMERCIO: integer (nullable = true)
 |-- SECTOR: string (nullable = true)
 |-- DIA: string (nullable = true)
 |-- FRANJA_HORARIA: string (nullable = true)
 |-- IMPORTE: double (nullable = true)
 |-- NUM_OP: integer (nullable = true)



In [ ]:
df_cards.createOrReplaceTempView("TCARDS")
df_weather.createOrReplaceTempView("TWEATHER")

In [ ]:
import json

**Propuestas de consultas:**
* Días de sol con gastos
* Media de gastos por sector
* Meses de gastos
* Horas de mayor gasto en verano
* Compras en día de lluvia
* Compras con temperaturas extremas

**Creación de Dataframe para el JOIN**

In [ ]:
df_conteo = spark.sql("SELECT DIA, count(DIA) AS `ENTRADAS DE DATOS`, SUM(IMPORTE) AS `GASTOS` FROM TCARDS GROUP BY DIA ORDER BY DIA;")
df_conteo.createOrReplaceTempView("TDIAS")

**Pruebas para los join**

In [ ]:
df_cards.select('SECTOR', 'IMPORTE').groupby('SECTOR').sum('IMPORTE').show()

+-------------------+--------------------+
|             SECTOR|        sum(IMPORTE)|
+-------------------+--------------------+
|              HOGAR|    5811188.79000107|
|               AUTO|   2416929.799999998|
|       ALIMENTACION|2.8270662249997895E7|
|MODA Y COMPLEMENTOS|2.6734983769985665E7|
|            BELLEZA|  3187440.1600001054|
|       RESTAURACION|   5194634.310000212|
|              OTROS|  6954260.5400012825|
|              SALUD|1.1072741890000245E7|
|OCIO Y TIEMPO LIBRE|   4863853.710002758|
|         TECNOLOGIA|   3723798.409999931|
+-------------------+--------------------+



**Gastos por sector**

In [ ]:
dfsectores = spark.sql("SELECT SECTOR, ROUND(SUM(IMPORTE), 2) AS `GASTO TOTAL` FROM TCARDS GROUP BY SECTOR;")
dfsectores = dfsectores.toPandas()
sectores_json = dfsectores.to_json()
sectores_json = json.loads(sectores_json)
sectores_json

with open('/content/drive/MyDrive/DatosCollabBigData/sectores.json', 'w') as file:
    json.dump(sectores_json, file, indent=4)

**Meses de gastos**

In [ ]:
dfgastosMonth = spark.sql("SELECT MONTH(DIA) AS `MES`, ROUND(SUM(IMPORTE), 2) AS `IMPORTES` FROM TCARDS GROUP BY `MES` ORDER BY `MES`;")
dfmeses = dfgastosMonth.toPandas()
meses_json = dfmeses.to_json()
meses_json = json.loads(meses_json)
meses_json

with open('/content/drive/MyDrive/DatosCollabBigData/meses.json', 'w') as file:
    json.dump(meses_json, file, indent=4)

**Horas de mayor gasto en verano**

In [ ]:
dfgastosSummer = spark.sql("SELECT FRANJA_HORARIA, ROUND(SUM(IMPORTE), 2) AS `IMPORTE TOTAL` FROM TCARDS WHERE MONTH(DIA) BETWEEN 6 AND 8 GROUP BY FRANJA_HORARIA ORDER BY FRANJA_HORARIA ASC;")
dfsummer = dfgastosSummer.toPandas()
summer_json = dfsummer.to_json()
summer_json = json.loads(summer_json)
summer_json

with open('/content/drive/MyDrive/DatosCollabBigData/summer.json', 'w') as file:
    json.dump(summer_json, file, indent=4)

**Horas de mayor gasto en invierno**

In [ ]:
dfgastosWinter = spark.sql("SELECT FRANJA_HORARIA, ROUND(SUM(IMPORTE), 2) AS `IMPORTE TOTAL` FROM TCARDS WHERE MONTH(DIA) BETWEEN 1 AND 2 OR MONTH(DIA) = 12 GROUP BY FRANJA_HORARIA ORDER BY FRANJA_HORARIA ASC;")
dfwinter = dfgastosWinter.toPandas()
winter_json = dfwinter.to_json()
winter_json = json.loads(winter_json)
winter_json

with open('/content/drive/MyDrive/DatosCollabBigData/winter.json', 'w') as file:
    json.dump(winter_json, file, indent=4)

**Gastos con temperaturas altas**

In [ ]:
dfgastosTAltas = spark.sql("SELECT TDIAS.DIA, TWEATHER.TMed, ROUND(TDIAS.GASTOS, 2) AS `GASTOS` FROM TDIAS JOIN TWEATHER ON TDIAS.DIA = TWEATHER.FECHA WHERE TWEATHER.TMed >= 27 GROUP BY TDIAS.DIA, TWEATHER.TMed, TDIAS.GASTOS ORDER BY TDIAS.DIA;")
dftaltas = dfgastosTAltas.toPandas()
taltas_json = dftaltas.to_json()
taltas_json = json.loads(taltas_json)
taltas_json

with open('/content/drive/MyDrive/DatosCollabBigData/taltas.json', 'w') as file:
    json.dump(taltas_json, file, indent=4)

AnalysisException: ignored